# Feb 12, 2026: visualize membership matrices 
on brain, per solution mode

conda env: gt

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

# import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.ANALYSIS = 'trial-end'
# args.WINDOW = np.arange(5, 8+1)
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub'

BASE_path = f'/home/govindas/lab-data/aba'
PARC_DESC = f'NEWMAX_ROIs_final_gm_100_2mm' #f'NEWMAX_ROIs_final_gm_104_2mm' #f'ABA_ROIs_final_gm_36'
ROI_path = (
    f'{BASE_path}/{PARC_DESC}'
)
os.makedirs(ROI_path, exist_ok=True)
TS_path = f'{ROI_path}/roi-timeseries'
os.makedirs(TS_path, exist_ok=True)

ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/analysis-{args.ANALYSIS}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/cond-*/graphs'
SBM_path = f'{ROI_RESULTS_path}/cond-*/model-fits'
ESTIM_path = f'{ROI_RESULTS_path}/cond-*/estimates/individual'

In [4]:
args.num_rois = 100

In [5]:
parcels_img = ants.image_read(f'{BASE_path}/../ROI_mask/{PARC_DESC}.nii.gz')

try:
    roi_table = pd.read_csv(f'{BASE_path}/../ROI_mask/{PARC_DESC}_Info.txt', sep='\t')
    roi_labels = roi_table['Index'].to_list()
except:
    roi_labels = np.arange(1, args.num_rois+1)

In [6]:
roi_table

,ROI_name,Index,num_voxels
0,R_med._OFC,1,407
1,L_med._OFC,2,339
2,R_lat._OFC,3,56
3,L_lat._OFC,4,62
4,M_vmPFC1,5,67
...,...,...,...
95,Hypothalamus_PH,99,10
96,dlPAG,101,11
97,dmPAG,102,13
98,lPAG,103,13


In [7]:
parcels_img

ANTsImage (RPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (91, 109, 91)
	 Spacing    : (2.0, 2.0, 2.0)
	 Origin     : (-90.0, 126.0, -72.0)
	 Direction  : [ 1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [8]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def marginal_to_nifti(args, parcels_img: ants, X, mode_id, folder, level=-1):
    parcels_vol = parcels_img.numpy()
    parcels_vol = np.round(parcels_vol, decimals=0)

    os.makedirs(folder, exist_ok=True)
    args.num_rois, args.num_comms = X.shape

    in_files = []
    for idx_comm in range(args.num_comms):
        x = X[:, idx_comm]
        x_img = np.zeros_like(parcels_vol)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels_vol == roi) * (x[idx])
        
        file = f'{folder}/mode-{mode_id}_comm-{idx_comm}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    if level == -1:
        out_file = f'{folder}/mode-{mode_id:02d}.nii.gz'
    else:
        out_file = f'{folder}/mode-{mode_id:02d}_level-{level}.nii.gz'
    concatenate(in_files, out_file)
    
    # Retag the view and space so AFNI overlays it correctly
    # Use quotes around filename just in case of spaces in path
    os.system(f"3drefit -view tlrc -space MNI '{out_file}'")
    return None

In [9]:
args.dc, args.sbm = False, 'h'

args.nested = True if args.sbm in ['h'] else False

args.force_niter = 40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['m', 'a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-nd-h'

In [10]:
def get_membership_matrix(num_rois, df, col='pi'):
    pis = [np.zeros((num_rois, 1)) if np.isnan(pi).all() else pi for pi in df[col]]

    num_modes = len(df)
    num_comms = np.max([pi.shape[-1] for pi in pis])
    num_rois = num_rois
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode, pi in enumerate(pis):
        M[:, idx_mode, :pi.shape[-1]] = pi
    
    return M

In [11]:
marginals_files = sorted(glob.glob(f'{ESTIM_path}/sub-*/partition-modes-group-aligned/{SBM}/desc-mem-mats.pkl', recursive=True))
marginals_df = []
for sbm_file in marginals_files:
    with open(f'{sbm_file}', 'rb') as f:
        row = pickle.load(f)
    marginals_df += [row]
marginals_df = pd.concat(marginals_df).reset_index(drop=True)
mode_ids = list(chain.from_iterable([list(range(count)) for count in marginals_df['sub'].value_counts().sort_index().to_list()]))
marginals_df['mode_id'] = mode_ids
marginals_df

,sub,cond,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,omega,sigma,mode_id
0,ABA602,highR,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1....","[[0.9711538461538461, 0.0, 0.01923076923076923...","[[0.9156804733727811, 0.0, 0.08431952662721895...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.104267,0.052367,0
1,ABA602,highR,sbm-nd-h,"[[0.9879518072289156, 0.0, 0.0, 0.0, 0.0120481...","[[0.9766294092030773, 0.0, 0.01858034547829873...","[[0.9288196478758699, 0.0, 0.07118035212413014...","[[0.9898313782679815, 0.0, 0.01016862173201859...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...",0.082627,0.066361,1
2,ABA602,highR,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1....","[[0.5416666666666667, 0.01388888888888889, 0.4...","[[0.998070987654321, 0.0019290123456790127, 0....","[[0.9998070987654322, 0.00019290123456790125],...","[[1.0], [1.0], [1.0], [1.0000000000000002], [1...","[[1.0], [1.0], [1.0], [1.0000000000000002], [1...","[[1.0], [1.0], [1.0], [1.0000000000000002], [1...","[[1.0], [1.0], [1.0], [1.0000000000000002], [1...",0.072293,0.054583,2
3,ABA602,highR,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1....","[[0.4925373134328358, 0.0, 0.5074626865671642,...","[[0.9443729834218412, 0.04092441020702937, 0.0...","[[0.9856992686660718, 0.0069494281483634774, 0...","[[0.9992278413168486, 0.0007721586831514974], ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.066533,0.063806,3
4,ABA602,highR,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1....","[[0.4, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.9747985347985348, 0.01904761904761905, 0.0...","[[0.9973727422003285, 0.002627257799671593], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.064987,0.070884,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,ABA607,lowT,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.009137,0.070599,73
407,ABA607,lowT,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....","[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, ...","[[0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.006176,0.102052,74
408,ABA607,lowT,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1....","[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.004575,0.070962,75
409,ABA607,lowT,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [0.

In [12]:
cols = [col for col in  list(marginals_df.columns) if 'pi_' in col]
cols

['pi_0_aligned',
 'pi_1_aligned',
 'pi_2_aligned',
 'pi_3_aligned',
 'pi_4_aligned',
 'pi_5_aligned',
 'pi_6_aligned',
 'pi_7_aligned']

In [13]:
# SOFT MARGINALS
soft_marginals_df = []
for (sub, cond), group in marginals_df.groupby(['sub', 'cond']):
    omegas = group['omega'].to_list()
    dct = {'sub': [sub], 'cond':[cond], 'sbm': [SBM]}
    for col in cols:
        M = get_membership_matrix(args.num_rois, group, col=col)
        SCs = np.average(M, axis=1, weights=omegas) # soft-comms.
        SCs = np.round(SCs, decimals=3)
        dct[col] = [SCs]
    soft_marginals_df += [pd.DataFrame(dct)]
soft_marginals_df = pd.concat(soft_marginals_df).reset_index(drop=True)
soft_marginals_df

,sub,cond,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned
0,ABA602,highR,sbm-nd-h,"[[0.99, 0.0, 0.0, 0.0, 0.008, 0.0, 0.0, 0.0, 0...","[[0.731, 0.01, 0.256, 0.0, 0.001, 0.0, 0.0, 0....","[[0.945, 0.008, 0.047, 0.0, 0.0, 0.0, 0.0], [0...","[[0.995, 0.001, 0.004], [0.995, 0.001, 0.004],...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
1,ABA602,highT,sbm-nd-h,"[[0.837, 0.0, 0.0, 0.0, 0.162, 0.0, 0.0, 0.0, ...","[[0.058, 0.164, 0.777, 0.0, 0.001, 0.0, 0.0, 0...","[[0.953, 0.044, 0.002, 0.0, 0.0, 0.0, 0.0], [0...","[[0.996, 0.003, 0.001], [0.994, 0.005, 0.001],...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
2,ABA602,lowR,sbm-nd-h,"[[0.999, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.254, 0.601, 0.142, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.934, 0.055, 0.011, 0.0, 0.0, 0.0, 0.0], [0...","[[0.99, 0.009, 0.001], [0.995, 0.004, 0.001], ...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
3,ABA602,lowT,sbm-nd-h,"[[0.0, 0.981, 0.0, 0.0, 0.0, 0.0, 0.018, 0.0, ...","[[0.235, 0.136, 0.628, 0.001, 0.0, 0.0, 0.0, 0...","[[0.975, 0.019, 0.005, 0.0, 0.0, 0.0, 0.0], [0...","[[0.996, 0.004, 0.0], [0.995, 0.005, 0.0], [0....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
4,ABA603,highR,sbm-nd-h,"[[0.904, 0.096, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.906, 0.001, 0.093, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.976, 0.003, 0.021, 0.0, 0.0, 0.0, 0.0], [0...","[[0.999, 0.0, 0.001], [0.999, 0.0, 0.001], [0....","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
5,ABA603,highT,sbm-nd-h,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.556, 0.0, 0.444, 0.0, 0.0, 0.0, 0.0, 0.0],...","[[0.969, 0.012, 0.019, 0.0, 0.0, 0.0, 0.0], [0...","[[0.998, 0.002, 0.001], [0.998, 0.002, 0.001],...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
6,ABA603,lowR,sbm-nd-h,"[[0.001, 0.996, 0.0, 0.0, 0.0, 0.0, 0.003, 0.0...","[[0.492, 0.055, 0.454, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.964, 0.02, 0.015, 0.0, 0.0, 0.001, 0.0], [...","[[0.997, 0.002, 0.001], [0.996, 0.003, 0.001],...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
7,ABA603,lowT,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.294, 0.19, 0.516, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.919, 0.061, 0.02, 0.0, 0.0, 0.0, 0.0], [0....","[[0.993, 0.005, 0.002], [0.993, 0.005, 0.002],...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
8,ABA604,highR,sbm-nd-h,"[[0.016, 0.984, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.432, 0.004, 0.564, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.976, 0.009, 0.015, 0.0, 0.0, 0.0, 0.0], [0...","[[0.996, 0.001, 0.003], [0.996, 0.001, 0.003],...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], 

In [14]:
# for idx, row in marginals_df.iterrows():
#     for col in cols[:2]:
#         pi = row[col]
#         if np.any(np.isnan(pi)): continue
#         mode_id = row['mode_id']
#         sub = row['sub']
#         cond = row['cond']
#         level = col.replace('pi_', '').replace('_aligned', '')
#         if level == 'aligned': level = -1

#         folder = (
#             f'{ROI_RESULTS_path}/cond-{cond}/estimates/individual'
#             f'/sub-{sub}/marginal-visuals/nii/{SBM}'
#         )
#         os.makedirs(folder, exist_ok=True)
#         print(sub, cond, mode_id, level)
#         marginal_to_nifti(args, parcels_img, pi, mode_id, folder, level)
#     #     break
#     # break

In [15]:
for idx_row, row in soft_marginals_df.iterrows():
    sub, cond = row[['sub', 'cond']]
    for col in cols[:2]:
        M = row[col]
        
        level = col.replace('pi_', '').replace('_aligned', '')
        
        folder = (
            f'{ROI_RESULTS_path}/cond-{cond}/estimates/individual'
            f'/sub-{sub}/expected-marginal-visuals/nii/{SBM}'
        )
        os.makedirs(folder, exist_ok=True)
        print(sub, cond, col, level)
        marginal_to_nifti(args, parcels_img, row[col], 0, folder, level)
    #     break
    # break

ABA602 highR pi_0_aligned 0
260212-15:45:43,642 nipype.interface INFO:
	 stderr 2026-02-12T15:45:43.641984:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
260212-15:45:43,643 nipype.interface INFO:
	 stderr 2026-02-12T15:45:43.643434:++ WARNING: file /home/govindas/.afni.log is now 102671531 (103 million) bytes long!
260212-15:45:43,646 nipype.interface INFO:
	 stderr 2026-02-12T15:45:43.646337:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
260212-15:45:43,647 nipype.interface INFO:
	 stderr 2026-02-12T15:45:43.647829:*+ WARNING: Set TR of output dataset to 1.0 s
260212-15:45:43,912 nipype.interface INFO:
	 stderr 2026-02-12T15:45:43.912533:++ elapsed time = 0.3 s


++ 3drefit: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: RW Cox
++ WARNING: file /home/govindas/.afni.log is now 102674917 (103 million) bytes long!
++ Processing AFNI dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-highR/estimates/individual/sub-ABA602/expected-marginal-visuals/nii/sbm-nd-h/mode-00_level-0.nii.gz
*+ WARNING: Changing the space of an ORIG view dataset may cause confusion!
*+ WARNING:  NIFTI copies will be interpreted as TLRC view (not TLRC space).
*+ WARNING:  Consider changing the view of the dataset to TLRC view also
 + changing dataset view code
 + loading and re-writing dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-highR/estimates/individual/sub-ABA602/expected-margina

ABA602 highR pi_1_aligned 1
260212-15:45:45,847 nipype.interface INFO:
	 stderr 2026-02-12T15:45:45.847089:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
260212-15:45:45,848 nipype.interface INFO:
	 stderr 2026-02-12T15:45:45.848039:++ WARNING: file /home/govindas/.afni.log is now 102675254 (103 million) bytes long!
260212-15:45:45,853 nipype.interface INFO:
	 stderr 2026-02-12T15:45:45.853616:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
260212-15:45:45,855 nipype.interface INFO:
	 stderr 2026-02-12T15:45:45.855201:*+ WARNING: Set TR of output dataset to 1.0 s
260212-15:45:46,81 nipype.interface INFO:
	 stderr 2026-02-12T15:45:46.081276:++ elapsed time = 0.2 s


++ 3drefit: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: RW Cox
++ WARNING: file /home/govindas/.afni.log is now 102678083 (103 million) bytes long!
++ Processing AFNI dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-highR/estimates/individual/sub-ABA602/expected-marginal-visuals/nii/sbm-nd-h/mode-00_level-1.nii.gz
*+ WARNING: Changing the space of an ORIG view dataset may cause confusion!
*+ WARNING:  NIFTI copies will be interpreted as TLRC view (not TLRC space).
*+ WARNING:  Consider changing the view of the dataset to TLRC view also
 + changing dataset view code
 + loading and re-writing dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-highR/estimates/individual/sub-ABA602/expected-margina

ABA602 highT pi_0_aligned 0
260212-15:45:49,534 nipype.interface INFO:
	 stderr 2026-02-12T15:45:49.534476:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
260212-15:45:49,535 nipype.interface INFO:
	 stderr 2026-02-12T15:45:49.535735:++ WARNING: file /home/govindas/.afni.log is now 102678420 (103 million) bytes long!
260212-15:45:49,547 nipype.interface INFO:
	 stderr 2026-02-12T15:45:49.547611:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
260212-15:45:49,550 nipype.interface INFO:
	 stderr 2026-02-12T15:45:49.550192:*+ WARNING: Set TR of output dataset to 1.0 s
260212-15:45:50,1 nipype.interface INFO:
	 stderr 2026-02-12T15:45:50.001357:++ elapsed time = 0.5 s


...

++ 3drefit: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: RW Cox
++ WARNING: file /home/govindas/.afni.log is now 102835287 (103 million) bytes long!
++ Processing AFNI dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-lowR/estimates/individual/sub-ABA607/expected-marginal-visuals/nii/sbm-nd-h/mode-00_level-1.nii.gz
*+ WARNING: Changing the space of an ORIG view dataset may cause confusion!
*+ WARNING:  NIFTI copies will be interpreted as TLRC view (not TLRC space).
*+ WARNING:  Consider changing the view of the dataset to TLRC view also
 + changing dataset view code
 + loading and re-writing dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-lowR/estimates/individual/sub-ABA607/expected-marginal-

ABA607 lowT pi_0_aligned 0
260212-15:47:37,371 nipype.interface INFO:
	 stderr 2026-02-12T15:47:37.371771:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
260212-15:47:37,373 nipype.interface INFO:
	 stderr 2026-02-12T15:47:37.373125:++ WARNING: file /home/govindas/.afni.log is now 102835623 (103 million) bytes long!
260212-15:47:37,384 nipype.interface INFO:
	 stderr 2026-02-12T15:47:37.383974:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
260212-15:47:37,386 nipype.interface INFO:
	 stderr 2026-02-12T15:47:37.385978:*+ WARNING: Set TR of output dataset to 1.0 s
260212-15:47:37,853 nipype.interface INFO:
	 stderr 2026-02-12T15:47:37.853909:++ elapsed time = 0.5 s


++ 3drefit: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: RW Cox
++ WARNING: file /home/govindas/.afni.log is now 102841221 (103 million) bytes long!
++ Processing AFNI dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-lowT/estimates/individual/sub-ABA607/expected-marginal-visuals/nii/sbm-nd-h/mode-00_level-0.nii.gz
*+ WARNING: Changing the space of an ORIG view dataset may cause confusion!
*+ WARNING:  NIFTI copies will be interpreted as TLRC view (not TLRC space).
*+ WARNING:  Consider changing the view of the dataset to TLRC view also
 + changing dataset view code
 + loading and re-writing dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-lowT/estimates/individual/sub-ABA607/expected-marginal-

ABA607 lowT pi_1_aligned 1
260212-15:47:39,788 nipype.interface INFO:
	 stderr 2026-02-12T15:47:39.788216:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
260212-15:47:39,789 nipype.interface INFO:
	 stderr 2026-02-12T15:47:39.789413:++ WARNING: file /home/govindas/.afni.log is now 102841557 (103 million) bytes long!
260212-15:47:39,794 nipype.interface INFO:
	 stderr 2026-02-12T15:47:39.794832:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
260212-15:47:39,797 nipype.interface INFO:
	 stderr 2026-02-12T15:47:39.797499:*+ WARNING: Set TR of output dataset to 1.0 s
260212-15:47:40,11 nipype.interface INFO:
	 stderr 2026-02-12T15:47:40.010995:++ elapsed time = 0.2 s


++ 3drefit: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
++ Authored by: RW Cox
++ WARNING: file /home/govindas/.afni.log is now 102844099 (103 million) bytes long!
++ Processing AFNI dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-lowT/estimates/individual/sub-ABA607/expected-marginal-visuals/nii/sbm-nd-h/mode-00_level-1.nii.gz
*+ WARNING: Changing the space of an ORIG view dataset may cause confusion!
*+ WARNING:  NIFTI copies will be interpreted as TLRC view (not TLRC space).
*+ WARNING:  Consider changing the view of the dataset to TLRC view also
 + changing dataset view code
 + loading and re-writing dataset /home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_100_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-sub/cond-lowT/estimates/individual/sub-ABA607/expected-marginal-

In [16]:
col.split('_')

['pi', '1', 'aligned']